#### Biblioteki

In [1]:
from datetime import datetime
import datetime as dt

import glob

import numpy as np

import math


import os

import pandas as pd

# Bibiloteki sktime
import sktime
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.compose import TransformedTargetForecaster, EnsembleForecaster
from sktime.forecasting.fbprophet import Prophet
from sktime.forecasting.model_selection import (ForecastingRandomizedSearchCV, ForecastingGridSearchCV, SingleWindowSplitter)
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.model_selection._split import temporal_train_test_split

# metryki
from sklearn.metrics import mean_squared_error
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error, mean_squared_percentage_error,MeanSquaredError
from sktime.transformations.series.feature_selection import FeatureSelection
from sktime.transformations.series.outlier_detection import HampelFilter
from sktime.utils.plotting import plot_series
from sklearn.ensemble import RandomForestRegressor
import plotly.express as px
import plotly.graph_objects as go

from xgboost import XGBRegressor

In [2]:
import pickle

In [3]:
from utils import wykres, compute_metrics, convert_and_preprocess_daily_to_monthly, convert_weekly_to_daily, convert_weekly_to_daily_xgb, convert_weekly_to_monthly, convert_weekly_to_monthly_xgb, wczytaj_dane_oczyszczone, wczytaj_dane_oczyszczone_do_predykcji, get_most_important_cols, get_most_important_cols2, generate_confidence_intervals, wczytaj_dane_odstajace_do_predykcji, wczytaj_dane_inwentaryzacje_do_predykcji, wczytaj_dane_odstajace, wczytaj_dane_inwentaryzacje, wczytaj_dane_wszystkie, fh_predykcje

#### Funkcje

In [4]:
check = 'niestety'

TEST_SIZE_DICT = {
    "oczyszczone": 31,
    "odstajace": 7,
    "inwentaryzacje":7
}

START_DATE = '' 
END_DATE = ''

model_config = {
    "oczyszczone":
        {
            "ARIMAX":
                {
                    "sp": 4,
                },
            "XGBoost":
                {
                    "window_lengths": [4, 8, 12, 32],
                    "cv_window_length": 52,
                },
            "group_period": 'W'
        },
    "odstajace":
        {
            "ARIMAX":
                {
                    "sp": 1,
                },
            "XGBoost":
                {
                    "window_lengths": [2, 4, 6, 12],
                    "cv_window_length": 6,
                },
            "group_period": 'M'
        },
    "inwentaryzacje":
        {
            "ARIMAX":
                {
                    "sp": 1,
                },
            "XGBoost":
                {
                    "window_lengths": [2, 4, 6, 12],
                    "cv_window_length": 6,
                },
            "group_period": 'M'

        }
}

def get_data_reader(model_type, mode="train/test"):
    if mode == "train/test":
        if model_type == "oczyszczone":
            return wczytaj_dane_oczyszczone
        if model_type == "odstajace":
            return wczytaj_dane_odstajace
        if model_type == "inwentaryzacje":
            return wczytaj_dane_inwentaryzacje
    elif mode == "pred":
        if model_type == "oczyszczone":
            return wczytaj_dane_oczyszczone_do_predykcji
        if model_type == "odstajace":
            return wczytaj_dane_odstajace_do_predykcji
        if model_type == "inwentaryzacje":
            return wczytaj_dane_inwentaryzacje_do_predykcji
    else:
        raise ValueError(f"Mode {mode} not supported.")
        

def get_pred_y_and_y_test_for_xgb():
    read_data = get_data_reader(model_type=MODEL_TYPE, mode="train/test")
    y, X = read_data(start_date=START_DATE, end_date=END_DATE, store=STORE)
    
    if len(y) > 0:
        y_train, y_test, X_train, X_test = temporal_train_test_split(y, X, test_size=TEST_SIZE_DICT[MODEL_TYPE])

        fh = ForecastingHorizon(y_test.index, is_relative=False)

        best_model_test = pickle.load(open(f'models/test/{STORE}_{MODEL_TYPE}_best_model.pkl', 'rb'))
        selected_features_test = pickle.load(open(f'models/test/selected_features/{STORE}_{MODEL_TYPE}_features.pkl', 'rb'))

        if kamil == 'niestety':
            if type(best_model_test) == ForecastingRandomizedSearchCV:

                X_test.index = X_test.index.to_period(model_config[MODEL_TYPE]["group_period"])
                fh = ForecastingHorizon(X_test.index, is_relative=False)

                y_pred_test = best_model_test.predict(X=X_test[selected_features_test], fh=fh).rename(columns={'yhat': "VALUE"})
                y_pred_test.index = y_pred_test.index.to_timestamp(model_config[MODEL_TYPE]["group_period"])

            else:
                y_pred_test = best_model_test.predict(X=X_test[selected_features_test], fh=y_test.index) \
                                                    .rename(columns={'yhat': "VALUE"})
        else:
            y_pred_test = best_model_test.predict(X=X_test[selected_features_test],fh=y_test.index) \
                                                    .rename(columns={'yhat': "VALUE"})
    return y_test, y_pred_test

def _generate_confidence_intervals(preds, residuals, alpha = 0.1):
    ci = np.quantile(residuals, 1 - alpha)

    ci_interv = pd.DataFrame()
    if ci >= 0:
        ci_interv['upper'] = preds + ci
        ci_interv['lower'] = preds - ci
    else:
        ci_interv['upper'] = preds - ci
        ci_interv['lower'] = preds + ci

    return ci_interv

def generate_prediction_intervals(preds, fh, model = '', residuals=None , alpha = 0.1):
    
    if type(model) == AutoARIMA:
        y_pred_intervals = model.predict_interval(fh= fh, X=eval(f'X_{MODEL_TYPE}_do_predykcji'))
        y_pred_intervals.columns = ['lower','upper'] 
        #y_pred_intervals.columns = y_pred_intervals_arimax.columns.get_level_values(2)
        
    elif type(model) == Prophet:
        y_pred_intervals = model.predict_interval(fh= fh, X=eval(f'X_{MODEL_TYPE}_do_predykcji'))
        y_pred_intervals.columns = ['lower','upper']
        
    elif type(model) == ForecastingRandomizedSearchCV:
        y_test, y_pred_test = get_pred_y_and_y_test_for_xgb()
        residuals = y_test - y_pred_test
        y_pred_intervals = _generate_confidence_intervals(preds = preds, residuals = residuals) 
    
    return y_pred_intervals

`Lista ze sklepami`

In [7]:
MODEL_TYPES = ["oczyszczone", "odstajace", "inwentaryzacje"]
ALL_STORES = [...]


#### Zatrzymanie wyświetlania warningów

In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

#### Pozostałe funkcje

In [7]:
def fh_predykcje(start = '', freq = 'M'):
    if freq == 'M':
        periods = _
    elif freq == 'W':
        periods = _
    else:
        pass
    horyzont = ForecastingHorizon(pd.date_range(start=start, periods=periods, freq=freq), is_relative=False)
    return horyzont

def postprocess_pred(y_pred):
    y_pred['VALUE'] = y_pred['VALUE'].apply(lambda x: min(x,0))
    return y_pred

# Master petla

In [ ]:
pd.options.mode.chained_assignment = None
START_PREDICTION_DATE_DATE=''
preds = []
for STORE in ALL_STORES:
    print(STORE)
    ##### wczytywanie modeli ------------------------------

    # oczyszczone
    best_model_oczyszczone = pickle.load(open(f'models/pred/{STORE}_oczyszczone_best_model.pkl', 'rb'))
    selected_features_oczyszczone = pickle.load(open(f'models/pred/selected_features/{STORE}_oczyszczone_features.pkl', 'rb'))
    X_oczyszczone_do_predykcji = wczytaj_dane_oczyszczone_do_predykcji(store=STORE, start_date= START_PREDICTION_DATE_DATE)[selected_features_oczyszczone]

    # odstajace
    best_model_odstajace = pickle.load(open(f'models/pred/{STORE}_odstajace_best_model.pkl', 'rb'))
    selected_features_odstajace = pickle.load(open(f'models/pred/selected_features/{STORE}_odstajace_features.pkl', 'rb'))
    X_odstajace_do_predykcji = wczytaj_dane_odstajace_do_predykcji(store=STORE, start_date=START_PREDICTION_DATE_DATE)[selected_features_odstajace]

    # inwentaryzacje
    try:
        best_model_inwentaryzacje = pickle.load(open(f'models/pred/{STORE}_inwentaryzacje_best_model.pkl', 'rb'))
        selected_features_inwentaryzacje = pickle.load(open(f'models/pred/selected_features/{STORE}_inwentaryzacje_features.pkl', 'rb'))
        X_inwentaryzacje_do_predykcji = wczytaj_dane_inwentaryzacje_do_predykcji(store=STORE, start_date= START_PREDICTION_DATE_DATE)[selected_features_inwentaryzacje]
    except:
        pass

    ##### generowanie predykcji -----------------------------
    # oczyszczone
    
    MODEL_TYPE = 'oczyszczone'
    
    # Część, którą musi dodać Kamil    
    if type(best_model_oczyszczone) == ForecastingRandomizedSearchCV:

        for i in [X_oczyszczone_do_predykcji]:
            i.index = i.index.to_period('W')
        fh = ForecastingHorizon(X_oczyszczone_do_predykcji.index, is_relative=False)
        _y_oczyszczone_pred = best_model_oczyszczone.predict(X=X_oczyszczone_do_predykcji, fh=fh)
        _y_oczyszczone_pred = _y_oczyszczone_pred.rename(columns={'yhat': "VALUE"})
        _y_oczyszczone_pred.index = _y_oczyszczone_pred.index.to_timestamp('W')

    else:
        _y_oczyszczone_pred = best_model_oczyszczone.predict(X=X_oczyszczone_do_predykcji, fh=fh_predykcje(freq='W'))
        _y_oczyszczone_pred = _y_oczyszczone_pred.rename(columns={'yhat': "VALUE"})
    
    
    y_oczyszczone_pred_intervals = generate_prediction_intervals(_y_oczyszczone_pred, fh=fh_predykcje(freq='W'), model = best_model_oczyszczone)
    y_oczyszczone_pred_intervals = convert_weekly_to_monthly(y_oczyszczone_pred_intervals)
    
    y_oczyszczone_pred = convert_weekly_to_monthly(_y_oczyszczone_pred)
    
    # odstajace
    
    MODEL_TYPE = 'odstajace'
    
    if type(best_model_odstajace) == ForecastingRandomizedSearchCV:

        for i in [X_odstajace_do_predykcji]:
            i.index = i.index.to_period('M')
        fh = ForecastingHorizon(X_odstajace_do_predykcji.index, is_relative=False)
        _y_odstajace_pred = best_model_odstajace.predict(X=X_odstajace_do_predykcji, fh=fh)
        _y_odstajace_pred.index = _y_odstajace_pred.index.to_timestamp('M')

    else:
        _y_odstajace_pred = best_model_odstajace.predict(X=X_odstajace_do_predykcji, fh=fh_predykcje(freq='M'))

    y_odstajace_pred = _y_odstajace_pred.rename(columns={'yhat': "VALUE"})
    y_odstajace_pred_intervals = generate_prediction_intervals(y_odstajace_pred, fh=fh_predykcje(freq='M'), model = best_model_odstajace)

    # inwentaryzacje
    
    MODEL_TYPE = 'inwentaryzacje'
    
    try:
        if type(best_model_inwentaryzacje) == ForecastingRandomizedSearchCV:

            for i in [X_inwentaryzacje_do_predykcji]:
                i.index = i.index.to_period('M')
            fh = ForecastingHorizon(X_inwentaryzacje_do_predykcji.index, is_relative=False)
            _y_inwentaryzacje_pred = best_model_inwentaryzacje.predict(X=X_inwentaryzacje_do_predykcji, fh=fh)
            _y_inwentaryzacje_pred.index = _y_inwentaryzacje_pred.index.to_timestamp('M')

        else:
            _y_inwentaryzacje_pred = best_model_inwentaryzacje.predict(X=X_inwentaryzacje_do_predykcji, fh=fh_predykcje(freq='M'))

        y_inwentaryzacje_pred = _y_inwentaryzacje_pred.rename(columns={'yhat': "VALUE"})
        y_inwentaryzacje_pred_intervals = generate_prediction_intervals(y_inwentaryzacje_pred, fh=fh_predykcje(freq='M'), model = best_model_inwentaryzacje)
    except:
        y_inwentaryzacje_pred = None

    y_pred = postprocess_pred(y_oczyszczone_pred) + postprocess_pred(y_odstajace_pred) + postprocess_pred(y_inwentaryzacje_pred)
    y_pred["lower"] = y_oczyszczone_pred_intervals["lower"] + y_odstajace_pred_intervals["lower"] + y_inwentaryzacje_pred_intervals["lower"]
    y_pred["upper"] = y_oczyszczone_pred_intervals["upper"].apply(lambda x: min(x,0)) + \
            y_odstajace_pred_intervals["upper"].apply(lambda x: min(x,0)) + \
            y_inwentaryzacje_pred_intervals["upper"].apply(lambda x: min(x,0))
    y_pred["STORE"] = STORE

    preds.append(y_pred)

    print(f"{STORE} done!")

`Tworzenie plików z predykcjami`

In [ ]:
final_preds = pd.concat(preds)
final_preds['VALUE'] = round(final_preds['VALUE'], 2)
final_preds.index.name = 'DATE'
final_preds.to_csv("final_preds.csv")

final_preds

`Sprawdzenie czy nie ma braków wartości`

In [ ]:
final_preds[final_preds.isna().any(axis=1)]

----------------------------------------------------------------------------------